# Utilitários Unificados — Modelos, Datasets e Ruído

Este arquivo reúne três funções principais, com interface unificada e documentação clara:

1. `load_model_unificado(modelo, caminho, ...)`
   - Carrega qualquer modelo: `linear`, `mlp`, `lstm` (Keras) ou `tft` (PyTorch Forecasting).
   - Aceita arquivo/diretório EXATO ou uma pasta raiz para descoberta recursiva.

2. Carregamento de dados (Parquet)
   - `linear/mlp/lstm` → Parquet com `*.meta.json` contendo `x_dim`, `y_dim` (e `seq_len`, `lead` no LSTM).
   - `tft` → Parquet (padrão: retorna DataFrame; opcional: cria `TimeSeriesDataSet`).

3. `add_noise_features(obj, sigma, tipo, ...)`
   - Adiciona ruído GAUSSIANO somente nas FEATURES.
   - `tipo='tfdata'` → aplica em `tf.data.Dataset` (x,y).
   - `tipo='tft'` → aplica em batches de `TimeSeriesDataSet`/`DataLoader` (chaves `encoder_cont`/`decoder_cont`).

> Observação: O notebook foi simplificado para Parquet apenas (sem TFRecords).

## 1) Carregamento Unificado de Modelos

Contrato rápido:
- Entradas:
  - `modelo`: `linear` | `mlp` | `lstm` | `tft`
  - `caminho`: arquivo/diretório exato OU uma pasta para varredura recursiva
  - `prefer_exts` (opcional): lista de extensões a priorizar (ex.: `[".cpfg", ".ckpt"]` para TFT)
  - `allow_unsafe` (bool): permite desserialização insegura apenas para artefatos LOCAIS (Lambda em Keras)
- Saídas: `(obj_modelo, info)`
  - `obj_modelo`: instancia do modelo carregado (Keras ou TemporalFusionTransformer)
  - `info`: dicionário com metadados úteis (`path`, `backend`, `kind`)

In [1]:
# Carregamento dos modelos treinados
import torch
import json, os


def load_model(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ File not found: {path}")

    ext = os.path.splitext(path)[1].lower()
    try:
        # ✅ Tenta instanciar o TFT a partir do checkpoint do PyTorch Lightning
        try:
            from pytorch_forecasting.models import TemporalFusionTransformer
            import pytorch_lightning as pl  # noqa: F401 (garante dependências do Lightning)
            model = TemporalFusionTransformer.load_from_checkpoint(
                path,
                map_location=torch.device("cpu"),
                strict=False,
            )
            print("✅ TemporalFusionTransformer instanciado a partir do checkpoint.")
        except Exception as e:
            print(f"[WARN] Falha ao instanciar TFT via load_from_checkpoint: {e}")
            print("↩️  Fazendo fallback para torch.load (objeto bruto do checkpoint)...")
            model = torch.load(path, map_location="cpu")
            print("✅ PyTorch Lightning checkpoint carregado (raw object).")
    except Exception as e:
        raise ValueError(f"❌ Unsupported file extension: {ext} - Error: {e}")

    # === Load optional JSON config ===
    json_path = f"{os.path.splitext(path)[0]}.model.json"
    config = None
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            config = json.load(f)
        print(f"🧩 Loaded config: {json_path}")

    return model, config


## Carregando modelo TFT
tft, info_tft = load_model('./modelos/treinamento/TFT/tft/best.ckpt')

model_list = [
    (tft, info_tft)
]

/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv_torch/lib/python3.12/site-packages/pytorch_forecasting/models/base/_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv_torch/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv_torch/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


✅ TemporalFusionTransformer instanciado a partir do checkpoint.


## 2) Carregando preprocessadores

In [2]:
import glob
import pickle
from typing import Dict, Any, List, Tuple, Optional


# === TFT ===
tft_path = "./data/treinamento/preprocessor/tft_preproc.pkl"
with open(tft_path, "rb") as f:
    tft_preproc = pickle.load(f)

# === PREPROCESSORS DICT ===
preprocessors = {
    "tft": tft_preproc,
}

print("✅ Todos os preprocessadores carregados com sucesso.")


✅ Todos os preprocessadores carregados com sucesso.


# Funções helper para análise dos modelos

## Funções de coleta de dados

In [3]:

import pandas as pd
from preprocessor_torch import TFTPreprocessor
import numpy as np

def load_dataset_info(model_type: str, dataset_type: str, problem_name: str) -> Dict[str, Any]:
    info_path = f'./data/{problem_name}/{model_type}_dataset_{dataset_type}.meta.json'
    if not os.path.exists(info_path):
        raise FileNotFoundError(f"❌ Dataset info file not found: {info_path}")
    with open(info_path, 'r') as f:
        info = json.load(f)
    return info



def get_problem_df(country_list, problem_name) -> pd.DataFrame:
    # Instanciando preprocessadores
    dataset_info = load_dataset_info(problem_name=problem_name, model_type="tft", dataset_type="test")
    destino_dir = f'./data/{problem_name}'

    preproc = TFTPreprocessor(
        model_name="TFT",
        lag=240,
        lead=72,
        country_list=country_list,
        feature_cols=dataset_info['feature_cols'],
        target_cols=dataset_info['target_cols'],
        data_dir=destino_dir
    )
    # 👇 Pass the already loaded model here
    df = preproc.load_tft_dataset(
        split_name='test',
    )
    return df, dataset_info

## Funções de avaliação

In [4]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn

def avaliar_modelo_tft(model, dataset, titulo="Avaliação TFT",
                       problem_name="problema",
                       save_dir="./resultados/graficos",
                       show_plots=True):
    """
    Returns final dataframe df_full with:
        - X columns
        - pred columns
        - true columns
    Plus metrics & plots (similar to Keras evaluator).
    """

    device = next(model.parameters()).device
    dl = dataset.to_dataloader(batch_size=128, shuffle=False)

    X_list = []
    Y_pred_list = []
    Y_true_list = []

    model.eval()
    with torch.no_grad():
        for batch_x, batch_y in dl:

            # ------------------------
            # X: last decoder step
            # ------------------------
            dec_cont = batch_x["decoder_cont"]       # (B, pred_len, n_reals)
            X_last = dec_cont[:, -1, :]              # (B, n_reals)
            X_list.append(X_last.cpu().numpy())

            # ------------------------
            # y_true
            # ------------------------
            Y_true_list.append(batch_y[0].cpu().numpy())

            # ------------------------
            # y_pred
            # ------------------------
            batch_x_dev = {k: v.to(device) for k, v in batch_x.items()}
            out = model(batch_x_dev)

            # extract actual prediction tensor from Output object
            if isinstance(out, dict):
                out_tensor = out["prediction"]
            else:
                out_tensor = out.prediction

            Y_pred_list.append(out_tensor.cpu().numpy())

    # -----------------------------------------------
    # STACK EVERYTHING
    # -----------------------------------------------
    X_vals = np.concatenate(X_list, axis=0)
    Y_pred = np.concatenate(Y_pred_list, axis=0)
    Y_true = np.concatenate(Y_true_list, axis=0)

    # Make sure tensors are 2D (N, pred_len)
    if Y_pred.ndim == 3 and Y_pred.shape[-1] == 1:
        Y_pred = Y_pred.squeeze(-1)
    if Y_true.ndim == 3 and Y_true.shape[-1] == 1:
        Y_true = Y_true.squeeze(-1)

    # -----------------------------------------------
    # BUILD COLUMN NAMES
    # -----------------------------------------------
    real_cols = dataset.reals

    if len(real_cols) != X_vals.shape[1]:
        print("⚠️ Warning: len(dataset.reals) != n_features in decoder_cont.")
        real_cols = [f"feat_{i}" for i in range(X_vals.shape[1])]

    X_cols = real_cols  # final list of feature names for X_vals

    horizon = Y_pred.shape[1]
    target_name = dataset.target if isinstance(dataset.target, str) else "target"

    Y_pred_cols = [f"{target_name}_lead{i+1}_pred" for i in range(horizon)]
    Y_true_cols = [f"{target_name}_lead{i+1}_true" for i in range(horizon)]

    # Convert everything into DataFrames
    df_x = pd.DataFrame(X_vals, columns=X_cols)
    df_y_pred = pd.DataFrame(Y_pred, columns=Y_pred_cols)
    df_y_true = pd.DataFrame(Y_true, columns=Y_true_cols)

    # Merge horizontally
    df_full = pd.concat([df_x, df_y_pred, df_y_true], axis=1)

    # =====================================================================
    # =====================================================================
    #                     🟩 ADD: GENERAL METRICS
    # =====================================================================
    # =====================================================================

    yt_all = df_y_true.values.reshape(-1)
    yp_all = df_y_pred.values.reshape(-1)
    diff = yp_all - yt_all

    mae = float(np.mean(np.abs(diff)))
    mse = float(np.mean(diff ** 2))
    rmse = float(np.sqrt(mse))

    if np.std(yt_all) > 0 and np.std(yp_all) > 0:
        corr = float(np.corrcoef(yt_all, yp_all)[0, 1])
    else:
        corr = float("nan")

    ss_res = np.sum((yt_all - yp_all) ** 2)
    ss_tot = np.sum((yt_all - np.mean(yt_all)) ** 2)
    r2 = float(1 - ss_res / ss_tot) if ss_tot > 0 else float("nan")

    resultados = dict(mae=mae, mse=mse, rmse=rmse,
                      correlacao_pearson=corr, r2=r2)

    # =====================================================================
    #                     🟧 ADD: PER-LEAD METRICS
    # =====================================================================
    per_lead = {}

    for true_col, pred_col in zip(Y_true_cols, Y_pred_cols):
        t = df_full[true_col].values
        p = df_full[pred_col].values
        d = p - t

        mae_i = float(np.mean(np.abs(d)))
        mse_i = float(np.mean(d ** 2))
        rmse_i = float(np.sqrt(mse_i))

        if np.std(t) > 0 and np.std(p) > 0:
            corr_i = float(np.corrcoef(t, p)[0, 1])
        else:
            corr_i = float("nan")

        ss_res = np.sum((t - p) ** 2)
        ss_tot = np.sum((t - np.mean(t)) ** 2)
        r2_i = float(1 - ss_res / ss_tot) if ss_tot > 0 else float("nan")

        metrics = [mae_i, mse_i, rmse_i, corr_i, r2_i]

        for k, v in zip(
                ["mae", "mse", "rmse", "correlacao_pearson", "r2"],
                metrics):
            per_lead.setdefault(k, []).append(v)

    # =====================================================================
    #                     🟦 OUTPUT DIR
    # =====================================================================
    output_dir = os.path.join(save_dir, problem_name, "TFT")
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # =====================================================================
    #                     🟪 PLOT: OVERALL METRICS TABLE
    # =====================================================================
    num_items = list(resultados.items())

    fig, ax = plt.subplots(figsize=(8, 0.4 * len(num_items) + 1))
    ax.axis("off")
    ax.table(
        cellText=[[k, f"{v:.6f}"] for k, v in num_items],
        colLabels=["Métrica", "Valor"],
        loc="center",
    )
    plt.tight_layout()
    fig.savefig(os.path.join(output_dir, "overall_metrics.png"), dpi=150)
    if show_plots: plt.show()
    else: plt.close()

    # =====================================================================
    #                     🟫 PLOT: METRIC PER LEAD
    # =====================================================================
    for metric, vals in per_lead.items():
        xs = np.arange(1, len(vals) + 1)
        plt.figure(figsize=(9, 4))
        plt.plot(xs, vals, marker="o")
        plt.title(f"{titulo} — {metric.upper()} por Lead")
        plt.xlabel("Lead")
        plt.ylabel(metric.upper())
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        fp = os.path.join(output_dir, f"metric_{metric}.png")
        plt.savefig(fp, dpi=150)
        if show_plots: plt.show()
        else: plt.close()

    # =====================================================================
    #                     🟩 PLOT REAL vs PRED FOR ALL LEADS
    # =====================================================================

    # x-axis
    if "datetime" in df_full.columns:
        x_axis = df_full["datetime"]
    else:
        x_axis = np.arange(len(df_full))

    for i, (true_col, pred_col) in enumerate(zip(Y_true_cols, Y_pred_cols), start=1):

        plt.figure(figsize=(12, 5))
        plt.plot(x_axis, df_full[true_col], label=f"Real {true_col}", linewidth=2)
        plt.plot(
            x_axis, df_full[pred_col],
            label=f"Pred {pred_col}",
            linewidth=2,
            linestyle="--"
        )
        plt.grid(True)
        plt.legend()
        plt.title(f"{titulo} — Lead {i}")
        plt.tight_layout()

        fp = os.path.join(output_dir, f"lead{i}_series.png")
        plt.savefig(fp, dpi=150)

        if show_plots:
            plt.show()
        else:
            plt.close()


    # =====================================================================
    df_full.to_csv(os.path.join("./", "df_full.csv"), index=False)
    return {
        "resultados": resultados,
        "per_lead": per_lead,
        "df": df_full,
        "y_true": Y_true,
        "y_pred": Y_pred
    }


# N1A — Série Univariada (seq_len=72, lead=72)


Objetivo
- Prever 24 horas de carga à frente com janelas de 48 horas de histórico para um único país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1A/linear_dataset_{split}.parquet` + `linear_dataset_{split}.meta.json` → { x_dim, y_dim }
- Parquet (LSTM): `data/N1A/lstm_dataset_{split}.parquet` + `lstm_dataset_{split}.meta.json` → { seq_len=240, lead=72, x_dim, y_dim }


Modelos a comparar
- Linear, MLP, LSTM (Keras) e, opcionalmente, TFT.


Métricas e checks
- MAE, RMSE, MAPE.
- Checar: shapes conforme meta.json; ausência de NaNs; número de amostras > 0.


Visualizações sugeridas
- Boxplot de erro por horizonte; barras de MAE por modelo; curva MAE vs horizonte.


Notas
- As variantes A/B são obtidas reduzindo a janela/horizonte efetivos na avaliação a partir do dataset base (240/72).
- Padding (se houver) deve usar sentinela fixo para permitir mascaramento e ruído seletivo.

In [5]:
# Carregamento dos dataset N1A (Parquet)

# # linear/mlp (iguais)
# ds_linear, linear_info = get_problem_df(
#     model_type='linear',
#     lag=72,
#     lead=72,
#     country_list=['ES'],
#     problem_name='N1A'
# )

# # lstm
# ds_lstm, lstm_info = get_problem_df(
#     model_type='lstm',
#     lag=72,
#     lead=72,
#     country_list=['ES'],
#     problem_name='N1A'
# )

# tft
ds_tft, tft_info = get_problem_df(
    country_list=['ES'],
    problem_name='N1A'
)

## Avaliando dados

### Avaliando modelo TFT
avaliar_modelo_tft(
    model=tft,
    dataset=ds_tft,
    titulo="Avaliação TFT - N1A",
    problem_name="N1A",
    save_dir="./resultados/graficos",
    show_plots=False 
)

📦 TimeSeriesDataSet (test) criado com 1746 amostras.


{'resultados': {'mae': 3965.37109375,
  'mse': 23200034.0,
  'rmse': 4816.641360948519,
  'correlacao_pearson': 0.07682579313204049,
  'r2': -0.7804917097091675},
 'per_lead': {'mae': [3525.4150390625,
   3589.0751953125,
   3646.2470703125,
   3694.289794921875,
   3740.4345703125,
   3780.8720703125,
   3814.58740234375,
   3842.662353515625,
   3865.802978515625,
   3884.931396484375,
   3900.66259765625,
   3912.302978515625,
   3922.40625,
   3933.948974609375,
   3945.766357421875,
   3957.450927734375,
   3967.546142578125,
   3976.1708984375,
   3984.00634765625,
   3991.369384765625,
   3998.680419921875,
   4004.959228515625,
   4009.44384765625,
   4013.539794921875,
   4017.613525390625,
   4021.617919921875,
   4025.821044921875,
   4028.48046875,
   4029.241455078125,
   4028.867431640625,
   4027.814208984375,
   4027.02197265625,
   4026.28271484375,
   4025.313232421875,
   4024.1025390625,
   4021.77197265625,
   4019.29541015625,
   4018.388427734375,
   4018.5075683

# N1B — Série Univariada (seq_len=168, lead=48)


Objetivo
- Prever 48 horas de carga à frente com janelas de 168 horas de histórico para um único país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1B/linear_dataset_{split}.parquet` + meta { x_dim, y_dim }
- Parquet (LSTM): `data/N1B/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (opcional): `data/treinamento/tft_dataset_{split}.parquet`


Modelos a comparar
- Linear, MLP, LSTM, TFT (opcional).


Métricas e checks
- MAE, RMSE, MAPE; validação de shapes e ausência de NaNs.


Visualizações sugeridas
- Barras de MAE médio por modelo; curva de erro por horizonte.


Notas
- As variantes A/B são derivadas do dataset base (240/72) reduzindo janela/horizonte na avaliação, sem retreinar.

# N1C — Série Univariada (seq_len=240, lead=72)


Objetivo
- Pré-treino/treino com a janela de 240 horas e avaliar horizonte de 72 horas.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1C/linear_dataset_{split}.parquet` + meta { x_dim, y_dim }
- Parquet (LSTM): `data/N1C/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (opcional): `data/treinamento/tft_dataset_{split}.parquet`


Modelos a comparar
- Linear, MLP, LSTM, TFT (opcional).


Métricas e checks
- MAE, RMSE, MAPE; número de amostras por split; coerência entre seq_len/lead do meta e shapes efetivos.


Visualizações sugeridas
- Curva comparativa de MAE vs horizonte; top‑k modelos por MAE.


Notas
- Esta variante (C) é a base máxima de lookback e horizonte; A/B são obtidas por redução na avaliação.

# N2A — Múltiplos Países (seq_len=72, lead=24)


Objetivo
- Prever 24 horas com 72 horas de histórico, agrupando por país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2A/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2A/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` (colunas: _group_id=country, time_idx crescente por grupo, quantity_MW)


Modelos a comparar
- Linear, MLP, LSTM (podem exigir codificação/flatten por grupo);
- TFT (nativamente multi‑grupo).


Métricas e checks
- MAE/RMSE por país e globais; número de grupos; equilíbrio de amostras por grupo.


Visualizações sugeridas
- Barras de MAE por modelo; facetas por país; curva MAE vs horizonte.


Notas
- As variantes A/B/C partem do dataset base (240/72), aplicando janelas/horizontes reduzidos na avaliação.

# N2B — Múltiplos Países (seq_len=168, lead=48)


Objetivo
- Prever 48 horas com 168 horas de histórico, agrupado por país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2B/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2B/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` com `_group_id`, `time_idx`, target.


Modelos a comparar
- Linear, MLP, LSTM; TFT.


Métricas e checks
- MAE/RMSE por país e agregadas; distribuição de amostras por grupo.


Visualizações sugeridas
- Barras de MAE por modelo; linhas por horizonte; painel por país.


Notas
- Variantes A/B/C usam janelas/horizontes efetivos na avaliação; base: 240/72.

# N2C — Múltiplos Países (seq_len=240, lead=72)


Objetivo
- Prever 72 horas com 240 horas de histórico, agrupado por país. Esta variante é a base para reuso em A/B.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2C/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2C/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` (agrupado por `_group_id`).


Modelos a comparar
- Linear, MLP, LSTM, TFT.


Métricas e checks
- MAE, RMSE, MAPE; comparação por país; checagem de time_idx e integridade por grupo.


Visualizações sugeridas
- Curva MAE vs horizonte; ranking de modelos por país e global.


Notas
- Base usa seq_len=240 e lead=72; variações A/B podem ser avaliadas reduzindo janela no dataset sem retreino.

# N3 — Robustez a Ruído (sobre N2)

Objetivo
- Medir degradação de desempenho sob ruído gaussiano nas FEATURES (teste), mantendo rótulos intactos.

Configuração
- Conjuntos: use os datasets do N2 (A/B/C).
- Intensidades: σ ∈ {0.00, 0.01, 0.03, 0.05, 0.10}.
- Aplicação:
  - Keras/tf.data: `add_noise_features(ds, sigma, tipo='tfdata', pad_sentinel=-999.0)`.
  - TFT: `add_noise_features(tft_ds ou dataloader, sigma, tipo='tft', batch_size=..., train=False)`.

Métricas e checks
- MAE/RMSE por sigma; checar preservação de sentinela (TF) e invariância de Y.

Visualizações sugeridas
- Curvas MAE vs σ por modelo; heatmap de degradação por horizonte e sigma.

Notas
- Aplique ruído após normalização das features.
- Não altere o treino; apenas avaliação/benchmark.
